In [1]:
from wayne_utils import load_data, save_data

tests = load_data( "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/data/N3/test_data_shuffle.jsonl", "jsonl")
trains = load_data( "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/data/N3/train_data_shuffle.jsonl", "jsonl")
test_nl_list = [ tests[i]['prompt'] for i in range(len(tests))]
train_nl_list = [ trains[i]['prompt'] for i in range(len(trains))]
train_gql_list = [ trains[i]['content'] for i in range(len(trains))]

intention_label_test = load_data( "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/methods/MyMethod/N3/intentions/intention_extract/intentions_label_test.json", "json")
intention_label_train = load_data( "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/methods/MyMethod/N3/intentions/intention_extract/intentions_label_train.json", "json")

In [2]:
# 相关schem查找
n3_schema = load_data( "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/data/N3/edge_vertex.json", "json")
n3_tests_dict = load_data( "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/data/N3/test_data_shuffle.jsonl", "jsonl")
N3_trains_dict = load_data(  "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/data/N3/train_data_shuffle.jsonl", "jsonl")

nba_schema = n3_tests_dict[0]["schema"].split("\n\n")
nba_schema_dict = {
    "Node": {
        "player" : nba_schema[1],
        "team" : nba_schema[2],
        "bachelor" : nba_schema[3],
    },
    "Edge":{
        "like" : nba_schema[5],
        "serve" : nba_schema[6],
        "teammate" : nba_schema[7]
    }
}

disease_schema = n3_tests_dict[2]["schema"].split("\n\n") 
disease_schema_dict = {
    "Node": {
        "drug" : disease_schema[1],
        "disease" : disease_schema[2],
        "food" : disease_schema[3],
        "check" : disease_schema[4],
        "department" : disease_schema[5],
        "producer" : disease_schema[6],
        "symptom" : disease_schema[7],
        "cure" : disease_schema[8],
    },
    "Edge":{
        "accompany_with" : disease_schema[10],
        "cure_department" : disease_schema[11],
        "cure_way" : disease_schema[12],
        "recommend_eat" : disease_schema[13],
        "need_check" : disease_schema[14],
        "no_eat" : disease_schema[15],
        "recommend_drug" : disease_schema[16],
    }
}

potter_schema = n3_tests_dict[8]["schema"].split("\n\n") 
potter_schema_dict = {
    "Node": {
        "character" : potter_schema[1],
        "college" : potter_schema[2],
        "group" : potter_schema[3],
    },
    "Edge":{
        "belong_to" : potter_schema[5],
        "kindred" : potter_schema[6],
        "learn_from" : potter_schema[7]
    }
}

def get_align_schema( class_, gql):
    sub_schema = {
        "nba" : nba_schema_dict,
        "disease" : disease_schema_dict,
        "potter" : potter_schema_dict
    }
    target_schema = sub_schema[class_]
    flag = 0
    ret_list = []
    for key in target_schema.keys():
        for class_name in target_schema[key]:
            if class_name in gql.lower():
                ret_list.append( target_schema[key][class_name])
                flag = 1
    if flag == 0:
        if "vertex" in gql.lower() or "node" in gql.lower():
            ret_list.extend( target_schema["Node"])
            flag = 1
        if "edge" in gql.lower():
            ret_list.extend( target_schema["Edge"])
            flag = 1

    if ret_list != []:
        ret_str = "# this is the related schema of this graph\n" + "\n\n".join(ret_list)
    else:
        ret_str = "# this is the related schema of this graph\n" + "no schema found"
    return ret_str

### 阶段一：生成意图

In [7]:
Intentions_rec = """你是一位图数据库语言专家，为了实现将用户自然语言查询转换为图数据库查询语言的目标，你需要首先做到对用户查询的意图识别。
所谓意图识别，就是要根据用户的自然语言查询，判断用户希望查询的对象是什么，以及用户希望以怎样的形式返回查询结果。

首先，根据图数据库的设计哲学，我们定义用户查询的对象可以分为以下几种类型：
（1）节点：图中的实体，比如人、物、地点等。节点可以单独存在。
（2）关系：图节点之间的联系，比如人与人之间的关系、物与物之间的关系等。一个关系依赖于两个节点。
（3）路径：图中实体与关系的组合，是连通的多个节点和关系。路径可以是简单的两个节点之间的关系，也可以是复杂的多个节点之间的关系。
（4）子图：指定一组节点和边，查找一定Step之内的所有边和节点

其次，明确用户查询意图中的对象类型、对象定义和约束外，还需要明确用户希望以何种形式返回查询结果。主要有以下几种形式：
（1）对查询对象的聚合操作、取值操作等其他函数操作。
（2）对返回对象进行排序、去重、分页等操作，例如返回满足条件的节点的个数、对节点的属性值进行排序、限制返回数量为5等。
请你根据用户输入的自然语言查询，判断用户查询的对象类型和返回形式。并根据上述类定义以下列示例结构化字典形式返回结果：
{   
    '查询对象': 字典列表，是意图识别的核心，通常为一个列表，存放用户查询可能涉及的所有对象，包括节点对象（node）、边(edge)、路径(path)以及子图(sub_graph)四种对象，每个对象都是一个字典，包含必要信息。
    '查询约束': 列表。表示对涉及对象的一些约束，例如路径类型，属性约束。
    '返回格式': 字典，则表示可能希望的返回值格式，例如是否去重、是否排序、分页、聚合函数等。
}
此外，在该数据库任务中，id表示节点或边的id值，通常为与name属性值一致的字符串。label则是指节点或边的tag/type，这是Nebula数据库强schema的要素，代表类型。
=======================================================================
"""

Input_rec = """下面进行实操：
自然语言查询：{NL}
数据库相关schema: {Schema}
用户意图：
"""

In [8]:
# 构建一阶段prompt_list和ft
def get_prompt_rec( raw_list, intention_list, test = True):
    if len( raw_list ) != len(intention_list):
        raise Exception("长度不一致")
    ret_list = []
    for i in range(len(intention_list) ):
        intention = intention_list[i]
        nl = raw_list[i]["prompt"]
        gql = raw_list[i]["content"]
        if test:
            scheams_re = get_align_schema( raw_list[i]["class"], gql)
        else:
            schemas = raw_list[i]['schema']
            if "player(Tag)" in schemas:
                class_ = "nba"
                scheams_re = get_align_schema( class_, gql)
            elif "drug(Tag)" in schemas:
                class_ = "disease"
                scheams_re = get_align_schema( class_, gql)
            elif "character(Tag)" in schemas:
                class_ = "potter"
                scheams_re = get_align_schema( class_, gql)
            else:
                scheams_re = "no schema found"
        if not test:
            ret_list.append({
                "instruction": Intentions_rec,
                "input": Input_rec.format( NL = nl, Schema =scheams_re),
                "output": str(intention)
            })
        else:
            ret_list.append( Intentions_rec + Input_rec.format( NL = nl, Schema = scheams_re ))
    return ret_list

In [11]:
intention_label_test

[{'查询对象': [{'category': 'node',
    'label': ['player'],
    'age': '待查询',
    'id': '待查询'}],
  '查询约束': ['player.age >= 29.5'],
  '返回格式': {'distinct': False,
   'aggregation': ['id'],
   'order by': None,
   'limit': None,
   'skip': None}},
 {'查询对象': [{'category': 'edge',
    'label': ['like'],
    'start_node': {'label': ['player'], 'name': 'Tim Duncan'},
    'end_node': {'label': []}}],
  '查询约束': [],
  '返回格式': {'distinct': False,
   'aggregation': [],
   'order by': None,
   'limit': None,
   'skip': None}},
 {'查询对象': [{'category': 'node',
    'label': ['disease'],
    'name': '房室管畸形',
    'symptom': '待查询'}],
  '查询约束': [],
  '返回格式': {'distinct': False,
   'aggregation': [],
   'order by': None,
   'limit': None,
   'skip': None}},
 {'查询对象': [{'category': 'node',
    'label': ['disease'],
    'name': '遗传性耳瘘',
    'prevent': '待查询'}],
  '查询约束': [],
  '返回格式': {'distinct': False,
   'aggregation': [],
   'order by': None,
   'limit': None,
   'skip': None}},
 {'查询对象': [{'category': 'node

In [12]:
# 意图识别微调与prompt
rec_prompt_list = get_prompt_rec( tests, intention_label_test, test = True)    # dict_keys(['prompt', 'content', 'text_schema', 'schema', 'class', 'result'])
rec_ft_list = get_prompt_rec( trains, intention_label_train, test = False) 
save_data( rec_prompt_list, "prompt_ft/rec_prompt_list.pickle")
save_data( rec_ft_list, "prompt_ft/N3_rec_intention.json")

In [29]:
len(rec_ft_list)

3862

### 阶段二：基于意图生成GQL

In [3]:
Instruction = """你是一名Nebula数据库专家，精通nGQL查询语言。下面请你根据用户自然语言查询，结合意图识别结果，生成nGQL查询语句。
自然语言查询部分是原始的用户自然语言表述的查询需求；数据库schema部分将提供该数据库中的类型定义；正式查询部分是能够在Nebula数据库中执行的语法严谨的nGQL语句。
意图识别结果：对用户查询进行意图识别后得到的字典数据，里面包含了与数据库对齐的标签名和属性名，以及一些基本的nGQL组件。细节如下：
(1) 查询对象: 是意图识别的核心，通常为一个列表，存放用户查询可能涉及的所有对象，包括节点对象（node）、边(edge)、路径(path)以及子图(sub_graph)四种对象，每个对象都是一个字典，包含必要信息。
(2) 查询约束: 表示对涉及对象的一些约束，例如路径类型，属性约束。
(3) 返回格式: 则表示可能希望的返回值格式，例如是否去重、是否排序、分页、聚合函数等。
"""

Input = """下面进行实操：
自然语言查询：{NL}
数据库相关schema: {Schema}
用户意图：{FAKE}
正式查询："""


In [4]:
# 构建二阶段prompt_list和ft
def get_prompt( raw_list, intention_list, test = True):
    if len( raw_list ) != len(intention_list):
        raise Exception("长度不一致")
    ret_list = []
    for i in range(len(intention_list) ):
        # intention = transform_obj( intention_list[i] )
        intention = intention_list[i]
        nl = raw_list[i]["prompt"]
        gql = raw_list[i]["content"]
        if test:
            scheams_re = get_align_schema( raw_list[i]["class"], gql)
        else:
            schemas = raw_list[i]['schema']
            if "player(Tag)" in schemas:
                class_ = "nba"
                scheams_re = get_align_schema( class_, gql)
            elif "drug(Tag)" in schemas:
                class_ = "disease"
                scheams_re = get_align_schema( class_, gql)
            elif "character(Tag)" in schemas:
                class_ = "potter"
                scheams_re = get_align_schema( class_, gql)
            else:
                scheams_re = "no schema found"
        if not test:
            ret_list.append({
                "instruction": Instruction,
                "input": Input.format( NL = nl, Schema =scheams_re, FAKE = intention ),
                "output": gql
            })
        else:
            ret_list.append( Instruction + Input.format( NL = nl, Schema = scheams_re ,FAKE = intention ))
    return ret_list

In [5]:
# 基于完美意图生成GQL微调数据集
ft_list = get_prompt( trains, intention_label_train, test = False) 
save_data( ft_list, "prompt_ft/N3_intention_to_gql.json")

In [6]:
# 基于完美意图生成查询：prompt_list
prompt_list = get_prompt( tests, intention_label_test, test = True)    # dict_keys(['prompt', 'content', 'text_schema', 'schema', 'class', 'result'])
save_data( prompt_list, "prompt_ft/perfect_prompt_list.pickle")

In [21]:
# 基于未对齐意图生成查询：prompt_list
non_align_test_intentions = load_data( "prompt_ft/rec_predict_list_N3_intention_rec_Chinese-Mistral-7B-Instruct-v0.1-1.5epoch.pickle", "pickle")
prompt_list = get_prompt( tests, non_align_test_intentions, test = True)    # dict_keys(['prompt', 'content', 'text_schema', 'schema', 'class', 'result'])
save_data( prompt_list, "prompt_ft/rec_non_align_prompt_list.pickle")

In [5]:
# 基于对齐后意图生成查询：prompt_list
aligned_test_intentions = load_data( "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/methods/MyMethod/N3/intentions/intention_extract/intention_aligned_3.json", "json")
prompt_list = get_prompt( tests, aligned_test_intentions, test = True)    # dict_keys(['prompt', 'content', 'text_schema', 'schema', 'class', 'result'])
save_data( prompt_list, "prompt_ft/rec_aligned_prompt_list.pickle")

In [23]:
non_align_test_intentions

["{'查询对象': [{'category': 'node', 'label': ['player'], 'age': '待查询', 'id': '待查询'}], '查询约束': ['player.age >= 29.5'], '返回格式': {'distinct': False, 'aggregation': ['id'], 'order by': None, 'limit': None, 'skip': None}}",
 '{\'查询对象\': [{\'category\': \'edge\', \'label\': [\'like\'], \'start_node\': {\'label\': []}, \'end_node\': {\'label\': []}}], \'查询约束\': [\'id(n) == "Tim"\'], \'返回格式\': {\'distinct\': False, \'aggregation\': [], \'order by\': None, \'limit\': None, \'skip\': None}}',
 "{'查询对象': [{'category': 'node', 'label': ['disease'], 'name': '房室管畸形', 'symptom': '待查询'}], '查询约束': [], '返回格式': {'distinct': False, 'aggregation': [], 'order by': None, 'limit': None, 'skip': None}}",
 "{'查询对象': [{'category': 'node', 'label': ['disease'], 'name': '遗传性耳瘘', 'prevent': '待查询'}], '查询约束': [], '返回格式': {'distinct': False, 'aggregation': [], 'order by': None, 'limit': None, 'skip': None}}",
 "{'查询对象': [{'category': 'node', 'label': ['team'], 'node_id_list': ['Spurs'], 'name': '待查询'}], '查询约束': [], '返回格式